In [1]:
# Import dependancies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
# from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Get Colony Square Events URL
url = "https://www.poncecitymarket.com/events"
response = requests.get(url)

In [3]:
# Parse response
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
# Create arrays for use in dataframe
event_name = []
event_desc = []
event_date = []
event_frequency = []
event_link = []

In [5]:
# Find first 10 event divs
# Scope of project is to use only first 10 events for each venue
results = soup.find_all('article', class_='eventlist-event eventlist-event--upcoming eventlist-event--hasimg eventlist-hasimg')[:10]
print(results)

[<article class="eventlist-event eventlist-event--upcoming eventlist-event--hasimg eventlist-hasimg">
<a class="eventlist-column-thumbnail content-fill" data-animation-role="image" href="/events/role-call-ponce-litty-spark-it-2021-lhtck"><img alt="RoleCall Theater: Ponce Litty Spark It" class="eventlist-thumbnail" data-image="https://images.squarespace-cdn.com/content/v1/5f9c80c54b29580773375eeb/1628958633498-S3ZS6MXE4PTM2AKUEMF5/Ponce+Litty+Spark+It.png" data-image-dimensions="1600x900" data-image-focal-point="0.5,0.5" data-load="false" data-src="https://images.squarespace-cdn.com/content/v1/5f9c80c54b29580773375eeb/1628958633498-S3ZS6MXE4PTM2AKUEMF5/Ponce+Litty+Spark+It.png"/></a>
<div class="eventlist-column-date">
<div class="eventlist-datetag">
<div class="eventlist-datetag-inner">
<div class="eventlist-datetag-startdate eventlist-datetag-startdate--month">Sep</div>
<div class="eventlist-datetag-startdate eventlist-datetag-startdate--day">10</div>
<div class="eventlist-datetag-sta

In [6]:
# Loop through results
for result in results:
    
    # Get event title
    event_title = result.find('a', class_='eventlist-title-link').text
    
    # Retrieve event link
    # Then, go to event page and pull first paragraph to use as description
    event_page = result.find('a', class_="eventlist-column-thumbnail content-fill")['href']
    event_response = requests.get(f'https://www.poncecitymarket.com/{event_page}')
    soup2 = BeautifulSoup(event_response.text, 'html')
    ep_results = result.find('div', class_ = "eventlist-excerpt").text
    
    # Get event date
    event_date_f = result.find('time', class_='event-date').text
    
    # Finds where 
    try:
        index = event_date_f.find("September")
        event_date_clean = event_date_f[index:] 
    
    except:
        event_date_clean = event_date_f
    
    # Get event occurance
    # Since we are only using the first 10 events, they are
    # Guaranteed to be within the month of September. 
    try:
        index2 = event_date_f.find("September")
        event_freq = event_date_f[:index]
        event_freq = event_freq.replace("from", "")
        
    except:
        event_freq = "Daily/One Day Event"
    
    # Append variables to arrays
    event_name.append(event_title)
    event_link.append(f'https://www.poncecitymarket.com/{event_page}')
    event_desc.append(ep_results)
    event_date.append(event_date_clean)
    event_frequency.append(event_freq)

In [7]:
# Create df using created arrays
ponce_city_df = pd.DataFrame({
    "event_name": event_name,
    "event_description": event_desc,
    "event_date": event_date,
    "event_frequency": event_frequency,
    "event_link": event_link
})

In [8]:
ponce_city_df

,event_name,event_description,event_date,event_frequency,event_link
0,RoleCall Theater: Ponce Litty Spark It,Ponce Litty Spark-It Stand Up ComedyHosted by ...,"September 10, 2021","Friday,",https://www.poncecitymarket.com//events/role-c...
1,RoleCall Theater: Amped Up Comedy,Alyx Libby brings her laid-back hosting style ...,"September 11, 2021","Saturday,",https://www.poncecitymarket.com//events/role-c...
2,RoleCall Theater: Amped Up Comedy,Alyx Libby brings her laid-back hosting style ...,"September 11, 2021","Saturday,",https://www.poncecitymarket.com//events/role-c...
3,Atlanta run club x Allbirds meets every monday,Join Atlanta Run Club every Monday from 6:30-8...,"September 13, 2021","Monday,",https://www.poncecitymarket.com//events/allbir...
4,H&F Burger trivia night,Enjoy trivia night with H&F Burger all summer ...,"September 13, 2021","Monday,",https://www.poncecitymarket.com//events/trivia...
5,RoleCall Theater: The Improv Jam,It's an improv party!!! Grab tickets to jam wi...,"September 15, 2021","Wednesday,",https://www.poncecitymarket.com//events/role-c...
6,RoleCall Theater: Amped Up Comedy,Get AMPED UP for a new stand-up comedy show! L...,"September 18, 2021","Saturday,",https://www.poncecitymarket.com//events/role-c...
7,Atlanta run club x Allbirds meets every monday,Join Atlanta Run Club every Monday from 6:30-8...,"September 20, 2021","Monday,",https://www.poncecitymarket.com//events/allbir...
8,RoleCall Theater: The Improv Jam,It's an improv party!!! Grab tickets to jam wi...,"September 22, 2021","Wednesday,",https://www.poncecitymarket.com//events/role-c...
9,RoleCall Theater: Ponce Litty Spark It,Ponce Litty Spark-It Stand Up ComedyHosted by ...,"September 24, 2021","Friday,",https://www.poncecitymarket.com//events/role-c...


In [9]:
ponce_city_df.to_csv("ponce_city.csv")